# Una solución empresarial completa

## Ahora llevaremos nuestro proyecto del día 1 al siguiente nivel

### DESAFÍO EMPRESARIAL:

Crear un producto que genere un folleto para una empresa que se utilizará para posibles clientes, inversores y posibles reclutas.

Se nos proporcionará un nombre de empresa y su sitio web principal.

Consulte el final de este cuaderno para ver ejemplos de aplicaciones empresariales del mundo real.

Y recuerde: ¡siempre estoy disponible si tiene problemas o ideas! No dude en comunicarse conmigo.

In [1]:
# imports
# Si esto falla, verifica que esté ejecutándose desde un entorno "activado" con (llms) en el símbolo del sistema

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Inicialización y constantes and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

La clave de API parece buena


In [3]:
# La clase para representar una Página Web

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [4]:
frog = Website("https://cursos.frogamesformacion.com")
#print(frog.get_contents())
frog.links

['#main-content',
 '/',
 '/',
 '/users/sign_in',
 'https://cursos.frogamesformacion.com/pages/rutas',
 'https://cursos.frogamesformacion.com/pages/instructores',
 'https://cursos.frogamesformacion.com/pages/certificaciones',
 'https://cursos.frogamesformacion.com/collections',
 '/courses/agentes-ia',
 '/courses/agentes-ia',
 '/courses/domina-android-desde-cero-kotlin-compose',
 '/courses/domina-android-desde-cero-kotlin-compose',
 '/courses/unity-ui',
 '/courses/unity-ui',
 '/courses/matematicas-ml-3',
 '/courses/matematicas-ml-3',
 '/courses/flutter-firebase-ai',
 '/courses/flutter-firebase-ai',
 '/courses/google-classroom',
 '/courses/google-classroom',
 '/courses/ia-productividad',
 '/courses/ia-productividad',
 '/courses/bandas-sonoras-4',
 '/courses/bandas-sonoras-4',
 '/courses/musica-videojuegos',
 '/courses/musica-videojuegos',
 '/courses/bandas-sonoras-3',
 '/courses/bandas-sonoras-3',
 '/courses/fundamentos-teoria-musical',
 '/courses/fundamentos-teoria-musical',
 '/courses/b

## Primer paso: hacer que GPT-4o-mini determine qué enlaces son relevantes

### Usar una llamada a gpt-4o-mini para leer los enlaces en una página web y responder en JSON estructurado.
Debería decidir qué enlaces son relevantes y reemplazar los enlaces relativos como "/about" con "https://company.com/about".
Usaremos "one shot prompting" en las que proporcionamos un ejemplo de cómo debería responder en la solicitud.

Este es un excelente caso de uso para un LLM, porque requiere una comprensión matizada. Imagínate intentar programar esto sin LLMs analizando la página web: ¡sería muy difícil!

Nota al margen: existe una técnica más avanzada llamada "Salidas estructuradas" en la que requerimos que el modelo responda de acuerdo con una especificación. Cubrimos esta técnica en la Semana 8 durante nuestro proyecto autónomo de inteligencia artificial Agentic.

In [5]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}
"""

In [6]:
print(link_system_prompt)

Se te proporciona una lista de enlaces que se encuentran en una página web. Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí hay una lista de enlaces de la página web {website.url} - "
    user_prompt += "Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.\n"
    user_prompt += "Links (puede que algunos sean links relativos):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(frog))

Aquí hay una lista de enlaces de la página web https://cursos.frogamesformacion.com - Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.
Links (puede que algunos sean links relativos):
#main-content
/
/
/users/sign_in
https://cursos.frogamesformacion.com/pages/rutas
https://cursos.frogamesformacion.com/pages/instructores
https://cursos.frogamesformacion.com/pages/certificaciones
https://cursos.frogamesformacion.com/collections
/courses/agentes-ia
/courses/agentes-ia
/courses/domina-android-desde-cero-kotlin-compose
/courses/domina-android-desde-cero-kotlin-compose
/courses/unity-ui
/courses/unity-ui
/courses/matematicas-ml-3
/courses/matematicas-ml-3
/courses/flutter-firebase-ai
/courses/flutter-firebase-ai
/courses/google-classroom
/courses/google-classroom
/courses/ia-productividad
/courses/ia-productividad
/c

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
anthropic = Website("https://anthropic.com")
anthropic.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonne

In [11]:
get_links("https://anthropic.com")

{'links': [{'type': 'Página Acerca de',
   'url': 'https://www.anthropic.com/company'},
  {'type': 'Carreras', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'Página de Cursos', 'url': 'https://www.anthropic.com/learn'},
  {'type': 'Página de soluciones',
   'url': 'https://www.anthropic.com/solutions/education'},
  {'type': 'Página de Clientes', 'url': 'https://www.anthropic.com/customers'},
  {'type': 'Página de Investigación',
   'url': 'https://www.anthropic.com/research'},
  {'type': 'Página de Noticias', 'url': 'https://www.anthropic.com/news'}]}

In [12]:
get_links("https://cursos.frogamesformacion.com")

{'links': [{'type': 'Página de Inicio',
   'url': 'https://cursos.frogamesformacion.com'},
  {'type': 'Página de Rutas',
   'url': 'https://cursos.frogamesformacion.com/pages/rutas'},
  {'type': 'Página de Instructores',
   'url': 'https://cursos.frogamesformacion.com/pages/instructores'},
  {'type': 'Página de Certificaciones',
   'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'},
  {'type': 'Página de Nuestros Clientes',
   'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'},
  {'type': 'Página de Frogames para Empresas',
   'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'},
  {'type': 'Página de Premios',
   'url': 'https://cursos.frogamesformacion.com/pages/premios'},
  {'type': 'Página de Afiliados',
   'url': 'https://cursos.frogamesformacion.com/pages/afiliados'}]}

## Segundo paso: ¡crea el folleto!

Reúne todos los detalles en otro mensaje para GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://anthropic.com"))

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Página de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Página de Cursos', 'url': 'https://www.anthropic.com/learn'}, {'type': 'Página de Productos', 'url': 'https://www.anthropic.com/enterprise'}]}
Landing page:
Título de la Web:
Home \ Anthropic
Contenido de la Web:
Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Max plan
Team plan
Enterprise plan
Explore pricing
Download apps
Claude log in
News
Claude’s character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collaborate with Claude
AI agents
Coding
Customer support
Education
Financial services
Case studies
Hear from our customers
Research
Research
Overview
Economic Index
Claude model family
Claude Opus 4
Claude Sonnet 4
Claude Haiku 3.5
Research
Claude’s extended thinking
Commitments
Initiatives
Trans

In [22]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
    y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
    Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."

# O descomenta las líneas a continuación para obtener un folleto más humorístico: esto demuestra lo fácil que es incorporar el "tono":

# system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa \
# y crea un folleto breve, divertido y gracioso sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
#Incluye detalles sobre la cultura de la empresa, los clientes y los cursos/packs para futuros empleos si tienes la información."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncar si tiene más de 20.000 caracteres
    return user_prompt

In [18]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Página de la empresa', 'url': 'https://www.anthropic.com/company'}, {'type': 'Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Página de Cursos', 'url': 'https://www.anthropic.com/learn'}, {'type': 'Página de investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Página del equipo', 'url': 'https://www.anthropic.com/team'}, {'type': 'Página de soluciones', 'url': 'https://www.anthropic.com/solutions'}]}


'Estás mirando una empresa llamada: Anthropic\nAquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\nLanding page:\nTítulo de la Web:\nHome \\ Anthropic\nContenido de la Web:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nMax plan\nTeam plan\nEnterprise plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nEducation\nFinancial services\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude Opus 4\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina Sobre nosotros', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Cursos', 'url': 'https://www.anthropic.com/learn'}, {'type': 'Pagina de Clientes', 'url': 'https://www.anthropic.com/customers'}, {'type': 'Pagina Principal', 'url': 'https://www.anthropic.com/'}]}


```markdown
# Folleto de Anthropic

Bienvenido a Anthropic, una empresa dedicada a la investigación y desarrollo de inteligencia artificial centrada en la seguridad y el bienestar humano.

## **Sobre Nosotros**
Anthropic es una **Corporación de Beneficio Público** con sede en San Francisco. Nuestra misión es construir sistemas de inteligencia artificial que sean confiables, interpretables y controlables, promoviendo un enfoque responsable en el desarrollo de estas tecnologías revolucionarias. Creemos que la IA tiene el potencial de cambiar el mundo y trabajamos para garantizar que este cambio sea positivo.

### **Valores Fundamentales**
- **Actuar para el bien global**: Tomamos decisiones audaces para maximizar los resultados positivos para la humanidad a largo plazo.
- **Mantener luz y sombra**: Reconocemos tanto los riesgos como los beneficios de la IA, buscando un balance entre ambos.
- **Ser buenos con nuestros usuarios**: Fomentamos la generosidad y la amabilidad en todas nuestras interacciones.
- **Inspirar una competencia por la seguridad**: Nos esforzamos por establecer un estándar alto de seguridad en la IA.
- **Hacer lo simple que funciona**: Adoptamos un enfoque empírico a los problemas, prefiriendo soluciones efectivas.
- **Ser útil, honesto y inofensivo**: Fomentamos un ambiente de confianza y colaboración.
- **Poner la misión primero**: La misión compartida guía nuestras decisiones y colaboración.

## **Nuestros Productos**
Presentamos **Claude**, nuestra serie de modelos de IA que abarca desde Claude Opus 4 hasta Claude Haiku 3.5. Estos modelos están diseñados para ser herramientas efectivas en una variedad de campos, incluyendo:

- **Educación**
- **Servicio al cliente**
- **Finanzas**
- **Desarrollo de soporte técnico**

## **Cultura de la Empresa**
En Anthropic, promovemos una cultura colaborativa y diversa. Nuestro equipo está formado por investigadores y expertos en política, ingeniería y operaciones, con una variedad de experiencias que alimentan nuestra misión. Valoramos la salud y el bienestar de nuestros empleados, ofreciendo:

- Seguro de salud, dental y de visión
- Beneficios de fertilidad inclusivos
- 22 semanas de licencia parental pagada
- Flexibilidad en vacaciones y políticas de ausencia
- Apoyo integral para salud mental

## **Oportunidades de Carrera**
Buscamos personas apasionadas que deseen marcar la diferencia en el desarrollo seguro de la IA. Ofrecemos una serie de roles en diferentes disciplinas y beneficios competitivos, incluyendo:

- Paquetes salariales y de equidad competitivos
- Programas de jubilación con igualación de mercado
- Estipendios para educación y oficina en casa

## **Anthropic Academy**
Para aquellos interesados en iniciar una carrera en IA, visiten nuestra **Anthropic Academy**, donde encontrarán recursos de aprendizaje y formación en IA, herramientas y técnicas de seguridad. Estamos comprometidos a educar y preparar a la próxima generación de profesionales en IA.

## **Únete a Nosotros**
Si estás interesado en contribuir al futuro seguro de la inteligencia artificial, visita nuestra página de **Carreras** y descubre las oportunidades disponibles. ¡Ayúdanos a construir un futuro en el que la IA sirva al bienestar humano!

---

**Contacto**
Visita nuestro sitio web en [Anthropic](https://www.anthropic.com) para más información.
```

Este folleto en formato Markdown describe la empresa Anthropic, destacando su enfoque en la IA segura, su cultura corporativa, oportunidades laborales y su compromiso con la educación.

In [21]:
create_brochure("cooperativa antú", "https://antucoop.site")

Links encontrados: {'links': [{'type': 'Pagina Sobre nosotros', 'url': 'https://antucoop.site#aboutUs'}, {'type': 'Pagina de Servicios', 'url': 'https://antucoop.site#services'}, {'type': 'Pagina de LinkedIn', 'url': 'https://www.linkedin.com/company/ant%C3%BA-cooperativa-de-desarollo-de-software/'}, {'type': 'Pagina de Instagram', 'url': 'https://www.instagram.com/antucoop/'}, {'type': 'Pagina de Facebook', 'url': 'https://www.facebook.com/antucoop'}]}


# Folleto de la Cooperativa Antú

## Acerca de Nosotros
Antú es una cooperativa de software que se destaca por su trabajo comprometido y de calidad, centrado en el desarrollo de aplicaciones web y móviles a medida. Con un equipo diverso y con experiencia, nos adaptamos a las necesidades de nuestros clientes y ofreciendo soluciones efectivas en cada proyecto.

### Misión
Brindar desarrollos personalizados que proporcionen una experiencia intuitiva y de calidad al usuario.

### Visión
Adoptar y utilizar las últimas tecnologías y metodologías ágiles para garantizar servicios de excelencia.

### Valores
- **Transparencia**
- **Responsabilidad**
- **Confiabilidad**
- **Puntualidad**
- **Excelencia**
  
## Nuestros Servicios
En Antú, ofrecemos una variedad de servicios especializados, incluyendo:

1. **Desarrollo de Software**  
   Creamos software a medida, prestando especial atención a la comunicación de requerimientos para maximizar los tiempos de entrega.

2. **Infraestructura**  
   Implementamos CI/CD utilizando GitLab CI y Docker, y armamos la infraestructura de tu proyecto utilizando AWS.

3. **Consultoría**  
   Asesoramos en tecnologías de la información, gestionando proyectos de software y aplicando metodologías efectivas.

## Cultura de la Empresa
En Antú creemos en el aprendizaje continuo. Nuestro equipo se encuentra en constante capacitación porque nos apasiona lo que hacemos. Fomentamos un ambiente de trabajo colaborativo, donde la innovación y la mejora continua son fundamentales.

## Oportunidades Laborales
Estamos en búsqueda de personas talentosas que quieran unirse a nuestro equipo. Si estás interesado en formar parte de nuestra cooperativa, no dudes en contactarnos.

## Capacitación para Futuras Carreras
Si estás buscando prepararte para el mercado laboral en el área de desarrollo de software, ofrecemos formación continua en tecnologías emergentes y metodologías ágiles.

## Contáctanos
Si tienes un nuevo proyecto en mente y estás en búsqueda de ayuda, no dudes en contactarnos:
- **Teléfono:** (+54) 11 6116 4844
- **Email:** [cooperativantu@gmail.com](mailto:cooperativantu@gmail.com)

## Síguenos en Redes Sociales
- [Facebook](#)
- [Instagram](#)
- [LinkedIn](#)

¡Esperamos poder colaborar contigo pronto!

## Por último, una pequeña mejora

Con un pequeño ajuste, podemos cambiar esto para que los resultados se transmitan desde OpenAI,
con la animación de máquina de escribir habitual


In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina de la empresa', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Cursos', 'url': 'https://www.anthropic.com/learn'}, {'type': 'Pagina del Equipo', 'url': 'https://www.anthropic.com/team'}, {'type': 'Pagina de Clientes', 'url': 'https://www.anthropic.com/customers'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Eventos', 'url': 'https://www.anthropic.com/events'}]}


# Folleto de Anthropic

## Acerca de Anthropic
Anthropic es una **Public Benefit Corporation** con sede en San Francisco, comprometida con el desarrollo responsable de la inteligencia artificial (IA) para el beneficio a largo plazo de la humanidad. Nuestra misión es construir sistemas de IA seguros, confiables, interpretables y manejables.

## Nuestra Cultura Empresarial
En Anthropic, creemos que la seguridad de la IA es fundamental. Nuestra cultura se basa en los siguientes valores:

1. **Actuar por el bien global**: Tomamos decisiones que maximizan los resultados positivos para la humanidad.
2. **Sostener la luz y la sombra**: Reconocemos tanto los riesgos como los beneficios potenciales de la IA y trabajamos para mitigarlos.
3. **Ser buenos con nuestros usuarios**: Fomentamos la generosidad y la amabilidad en nuestras interacciones.
4. **Despertar una carrera hacia la cima en seguridad**: Nos esforzamos por liderar en la creación de sistemas de IA seguros.
5. **Hacer lo simple que funcione**: Adoptamos un enfoque empírico a los problemas, buscando soluciones simples.
6. **Ser útiles, honestos y inofensivos**: Mantenemos una comunicación directa y asumiendo buenas intenciones.
7. **Poner la misión en primer lugar**: Nuestro objetivo compartido guía nuestras decisiones y acciones.

## Nuestros Clientes
Anthropic trabaja con una variedad de clientes en sectores como educación, apoyo al cliente, y servicios financieros. Desarrollamos herramientas como **Claude**, nuestro modelo de IA, que sirve para facilitar soluciones personalizadas en diversas industrias.

## Carreras en Anthropic
En Anthropic, valoramos a nuestro equipo diverso, que incluye investigadores, ingenieros y expertos en políticas. Ofrecemos un entorno colaborativo y estimulante para que todos puedan contribuir a nuestra misión. 

### Beneficios para Empleados
- **Salud y Bienestar**: Cobertura sanitaria, dental y de visión, beneficios de fertilidad inclusivos, y apoyo a la salud mental.
- **Compensación**: Salarios competitivos, planes de jubilación con igualación, y opciones de acciones.
- **Beneficios Adicionales**: Estipendios de bienestar, beneficios de transporte, y apoyo para oficinas en casa.

### Oportunidades de Desarrollo
Ofrecemos la **Anthropic Academy**, donde los nuevos empleados y quienes deseen mejorarse pueden acceder a recursos para aprender sobre la IA y cómo construir con Claude.

## Compromiso con la Transparencia
Anthropic se compromete a ser transparentes en nuestras operaciones y a compartir regularmente nuestros resultados e investigaciones con el mundo, garantizando la confianza en el desarrollo de tecnologías seguras.

---

¿Estás interesado en ser parte de nuestra misión de construir una IA más segura y accesible? **[Únete a nosotros](http://anthropic.com/careers)** y contribuye a un futuro mejor.

In [25]:
stream_brochure("HuggingFace", "https://huggingface.co")

Links encontrados: {'links': [{'type': 'Pagina Sobre nosotros', 'url': 'https://huggingface.co/huggingface'}, {'type': 'Pagina de Empleos', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Pagina de Cursos', 'url': 'https://huggingface.co/learn'}, {'type': 'Pagina de Documentación', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face: La Comunidad de IA que Construye el Futuro

## Sobre la Empresa
Hugging Face es una plataforma innovadora que impulsa la colaboración en la comunidad de aprendizaje automático. Con más de 50,000 organizaciones -incluidos gigantes como Google, Microsoft y Amazon- utilizando su tecnología, Hugging Face se ha convertido en un pilar fundamental en el desarrollo de modelos de IA, conjuntos de datos y aplicaciones.

### Misión
Nuestra misión es democratizar el acceso al aprendizaje automático y construir herramientas de IA accesibles para todos. Cada contribución que hacemos se basa en nuestra visión de un futuro donde la IA es accesible y benéfica para todos.

### Cultura de la Empresa
Hugging Face se enorgullece de su comunidad inclusiva y colaborativa. Fomentamos un entorno donde todos los miembros pueden aprender, compartir sus conocimientos y contribuir al avance de la IA. Creemos que el trabajo en equipo y la diversidad son la clave para innovar.

---

## Productos y Servicios

### Modelos
Hugging Face ofrece acceso a **más de 1 millón de modelos** de machine learning en diferentes modalidades: texto, imágenes, audio y más. La plataforma incluye:

- **Transformers**: Modelos de última generación para PyTorch.
- **Diffusers**: Modelos de difusión avanzados.
- **Tokenizers**: Herramientas rápidas para tokenización.

### Datasets
Proporcionamos acceso a **más de 250,000 conjuntos de datos** para tareas de aprendizaje automático, facilitando así la investigación y desarrollo.

### Espacios
Hugging Face permite a los usuarios crear y ejecutar **aplicaciones de IA** en la nube, lo que facilita la implementación y el uso de modelos en aplicaciones reales.

### Soluciones Empresariales
Ofrecemos soluciones de **compute y Enterprise** con características avanzadas como seguridad a nivel empresarial, soporte dedicado y controles de acceso.

---

## Oportunidades de Empleo

Estamos en búsqueda de **talento apasionado** por la inteligencia artificial y el aprendizaje automático. Nuestras ofertas laborales abarcan diversas especialidades, desde investigación y desarrollo, hasta marketing y soporte. Puedes unirte a nosotros para contribuir a un futuro impulsado por IA.

---

## Aprendizaje y Desarrollo

Hugging Face también se dedica a la formación y desarrollo de futuros profesionales a través de varios cursos:

- **Curso de LLM**: Aprende sobre modelos de lenguaje grandes.
- **Curso de RL**: Introdúcete al aprendizaje por refuerzo.
- **Curso de Visión por Computadora**: Enfócate en ML para imágenes.
- **Curso de Audio**: Utiliza transformadores en datos de audio.

Estos cursos están diseñados para equipar a los participantes con habilidades prácticas para avanzar en sus carreras en IA.

---

## Conéctate con Nosotros

¿Listo para formar parte de la revolución en IA? **Únete a la comunidad Hugging Face** y colabora en la construcción del futuro de la inteligencia artificial. 

Visita [huggingface.co](https://huggingface.co) para obtener más información y explorar nuestros modelos, datasets y oportunidades laborales. 

**¡Construyamos juntos un futuro más inteligente y accesible!**

In [ ]:
stream_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

## Aplicaciones empresariales

En este ejercicio, ampliamos el código del día 1 para realizar múltiples llamadas a LLM y generar un documento.

En términos de técnicas, este es quizás el primer ejemplo de patrones de diseño de Agentic AI, ya que combinamos múltiples llamadas a LLM. Esto se abordará más en la semana 2 y luego volveremos a Agentic AI de manera importante en la semana 8, cuando construyamos una solución Agent completamente autónoma.

En términos de aplicaciones empresariales, generar contenido de esta manera es uno de los casos de uso más comunes. Al igual que con el resumen, esto se puede aplicar a cualquier vertical empresarial. Escriba contenido de marketing, genere un tutorial de producto a partir de una especificación, cree contenido de correo electrónico personalizado y mucho más. Explore cómo puede aplicar la generación de contenido a su negocio e intente crear un prototipo de prueba de concepto.